In [1]:
import nltk
nltk.download('stopwords')
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow.python.framework import ops
from tensorflow.contrib import learn

[nltk_data] Downloading package stopwords to /home/fatma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/fatma/anaconda2/envs/conda_python_env/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)


In [2]:
st = LancasterStemmer()

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " ", string)
    string = re.sub(r"\'ve", " ", string)
    string = re.sub(r"n\'t", " ", string)
    string = re.sub(r"n\'s", " ", string)
    string = re.sub(r"\'re", " ", string)
    string = re.sub(r"\'d", " ", string)
    string = re.sub(r"\'ll", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\//?", " ", string)
    string = re.sub(r"\d+", " ", string)
    string = re.sub(r"\$", " ", string)
    string = re.sub(r"\#", " ", string)
    return string.strip().lower()

In [4]:
def remove_stop(str):
    stop = set(stopwords.words('english'))
    lst = str.split()
    lst = [i for i in lst if i not in stop]
    return ' '.join(lst)

def stem (str):
    lst = str.split()
    lst = [st.stem(x) for x in lst]
    return ' '.join(lst)

def remove_unwanted_words(str):
    unwanted_words = ["httpaddress", "usrid", "D", "dd", "rt", "amp", "am", "pm", '``',
                      "''", "", "//", "\\", "\\'s", "\\?", "\?","http","httpaddresshttpaddresst", "cohttpaddressek",
                      "taksim","gezi", "park", "direngeziparki", "occupygezi", "istanbul", "turkish","turkey",
                      "protest","direngezipark","direnankara","geziparki", "protesters", "protests", "sat", "sun", "mon,",
                     "tue", "wed", "thu", "fri", "jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    lst = str.split(" ")
    lst = [i for i in lst if i not in unwanted_words]
    return ' '.join(lst)

def toLower(str):
    lst = str.split()
    lst = [i.lower() for i in lst]
    return ' '.join(lst)

def word_len (str):
    lst = str.split()
    lst = [i for i in lst if len(i)>1 and len(i) <7]
    return ' '.join(lst)

def sent_len (str):
    lst = str.split()
    if len(lst)>=3:
        return ' '.join(lst)

In [5]:
def load_data_and_labels_shuffled(positive_data_file, negative_data_file):
    """
    Loads MR polarity african_data from files, splits the african_data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load african_data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    positive_examples = [remove_stop(item) for item in positive_examples]
    #print(positive_examples[0])
    positive_examples = [toLower(item) for item in positive_examples]
    #rint(positive_examples[0])
    positive_examples = [clean_str(sent) for sent in positive_examples]
    #rint(positive_examples[0])
    #positive_examples = [stem(item) for item in positive_examples]
    #rint(positive_examples[0])
    positive_examples = [remove_unwanted_words(item) for item in positive_examples]
    positive_examples = [word_len(item) for item in positive_examples]
    positive_examples = [sent_len(item) for item in positive_examples]
    positive_examples = list(filter(None, positive_examples))
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    negative_examples = [toLower(item) for item in negative_examples]
    #print(negative_examples[0])
    negative_examples = [remove_stop(item) for item in negative_examples]
    #print(negative_examples[0])
    negative_examples = [clean_str(sent) for sent in negative_examples]
    #print(negative_examples[0])
    #ngative_examples = [stem(sent) for sent in negative_examples]
    #pint(negative_examples[0])
    negative_examples = [remove_unwanted_words(item) for item in negative_examples]
    negative_examples = [word_len(item) for item in negative_examples]
    #print(negative_examples[0])
    negative_examples = [sent_len(item) for item in negative_examples]

    negative_examples = list(filter(None, negative_examples))

    # Split by words
    x_text = positive_examples + negative_examples
    # x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [1 for _ in positive_examples]
    negative_labels = [0 for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

In [8]:
def bag_of_words_model(features, target):
  """A bag-of-words model. Note it disregards the word order in the text."""
  target = tf.one_hot(target, 2, 1, 0)
  features = tf.contrib.layers.bow_encoder(
      features, vocab_size=n_words, embed_dim=Embedding_size, scope="input_layer")
  hidden_layer1 = tf.contrib.layers.fully_connected(features, 100, scope="hidden_layer1")
  logits = tf.contrib.layers.fully_connected(hidden_layer1, 2, scope="output_layer",
      activation_fn=None)
  loss = tf.contrib.losses.softmax_cross_entropy(logits, target)
  train_op = tf.contrib.layers.optimize_loss(
      loss, tf.contrib.framework.get_global_step(),
      optimizer='Adam', learning_rate=0.01)
  return (
      {'class': tf.argmax(logits, 1),
       'prob': tf.nn.softmax(logits)},
      loss, train_op)

In [6]:
ops.reset_default_graph()

# Start a graph session
sess = tf.Session()

# Choose max text word length at 25
sentence_size = 10
min_word_freq = 3
Words_Features = 'words'
Embedding_size = 50
n_words = 0

In [7]:
tweets_text, tweets_y = load_data_and_labels_shuffled('Data/Turkish_tweets_CF_results_09_05_2018_prccd_pos.txt', 'Data/Turkish_tweets_CF_results_09_05_2018_prccd_neg.txt')

In [9]:
# Setup vocabulary processor
vocab_processor = learn.preprocessing.VocabularyProcessor(sentence_size , min_frequency=min_word_freq)

# Have to fit transform to get length of unique words.
vocab_processor.transform(tweets_text)
embedding_size = len([x for x in vocab_processor.transform(tweets_text)])
print("embedding size = ", embedding_size)

embedding size =  1333


In [10]:
# Split up data set into train/test
train_indices = np.random.choice(len(tweets_text), round(len(tweets_text) * 0.7), replace=False)
test_indices = np.array(list(set(range(len(tweets_text))) - set(train_indices)))
texts_train = [x for ix, x in enumerate(tweets_text) if ix in train_indices]
texts_test = [x for ix, x in enumerate(tweets_text) if ix in test_indices]
target_train = [x for ix, x in enumerate(tweets_y) if ix in train_indices]
target_test = [x for ix, x in enumerate(tweets_y) if ix in test_indices]


In [11]:
# Process vocabulary
texts_train = np.array(list(vocab_processor.fit_transform(texts_train)))
texts_test = np.array(list(vocab_processor.transform(texts_test)))
n_words = len(vocab_processor.vocabulary_)

In [12]:
print('Total words: %d' % n_words)

print("train size", len(texts_train))
print("positive samples", target_train.count([1]))
print("negative samples", target_train.count([0]))

print("test size", len(texts_test))
print("positive samples",  target_test.count([1]))
print("negative samples", target_test.count([0]))

n_words = len(vocab_processor.vocabulary_)
print('no. words', n_words)

Total words: 316
train size 933
positive samples 129
negative samples 804
test size 400
positive samples 51
negative samples 349
no. words 316


In [15]:
n_epoches = 400
batch_size = 50

In [16]:
def batch_generator(data, batch_size, shuffle=True):
    """Generate batches of data.

    Given a list of array-like objects, generate batches of a given
    size by yielding a list of array-like objects corresponding to the
    same slice of each input.
    """
    if shuffle:
        data = shuffle_aligned_list(data)

    batch_count = 0
    while True:
        if batch_count * batch_size + batch_size >= len(data[0]): #the tarining examples
            batch_count = 0

            if shuffle:
                data = shuffle_aligned_list(data)

        start = batch_count * batch_size
        end = start + batch_size
        batch_count += 1
        
        yield [d[start:end] for d in data] #for the instances and then for the labels

In [18]:
gen_batch = batch_generator([texts_train, target_train], batch_size)

In [ ]:
with tf.Session() as sess:
    for epoch in range(n_epoches):
        for iteration in range(len(texts_train) // batch_size):
            X_batch, y_batch = next(gen_batch)
            sess.run(trainin_op, feed_dict={X: X_batch, y:y y_batch})
            

In [13]:
classifier = learn.Estimator(model_fn=bag_of_words_model)
# Train and predict
classifier.fit(texts_train, target_train, steps=10000)
y_predicted = [ p['class'] for p in classifier.predict(texts_test, as_iterable=True)]

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_session_config': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_task_type': None, '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f91d070fc50>, '_keep_checkpoint_max': 5, '_is_chief': True, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_master': '', '_num_ps_replicas': 0, '_evaluation_master': '', '_task_id': 0, '_model_dir': '/tmp/tmp1234vp66', '_save_checkpoints_secs': 600, '_num_worker_replicas': 0}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for 

INFO:tensorflow:global_step/sec: 191.321
INFO:tensorflow:step = 5301, loss = 0.029727882 (0.523 sec)
INFO:tensorflow:global_step/sec: 193.328
INFO:tensorflow:step = 5401, loss = 0.029716458 (0.517 sec)
INFO:tensorflow:global_step/sec: 194.199
INFO:tensorflow:step = 5501, loss = 0.029690463 (0.515 sec)
INFO:tensorflow:global_step/sec: 193.181
INFO:tensorflow:step = 5601, loss = 0.029711075 (0.518 sec)
INFO:tensorflow:global_step/sec: 194.979
INFO:tensorflow:step = 5701, loss = 0.029687783 (0.513 sec)
INFO:tensorflow:global_step/sec: 194.543
INFO:tensorflow:step = 5801, loss = 0.029711979 (0.514 sec)
INFO:tensorflow:global_step/sec: 195.017
INFO:tensorflow:step = 5901, loss = 0.02976758 (0.513 sec)
INFO:tensorflow:global_step/sec: 192.818
INFO:tensorflow:step = 6001, loss = 0.029704448 (0.519 sec)
INFO:tensorflow:global_step/sec: 194.729
INFO:tensorflow:step = 6101, loss = 0.029690998 (0.514 sec)
INFO:tensorflow:global_step/sec: 190.386
INFO:tensorflow:step = 6201, loss = 0.029942164 (0.

NameError: name 'metrics' is not defined

In [14]:
from sklearn import metrics
score = metrics.accuracy_score(target_test, y_predicted)
auc = metrics.roc_auc_score (target_test, y_predicted)
print('Accuracy: {0:f}'.format(score))
print('auc: {0:f}'.format(auc))

Accuracy: 0.822500
auc: 0.647143
